#### Dados

1. Importação das bibliotecas utilizadas

In [1]:
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
import google.generativeai as genai
import plotly.graph_objects as go

/home/zairo-bastos/Documentos/transportes/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. Leitura dos datasets
- Dataset com os dados do supermercado

In [2]:
atacarejo = pd.read_csv("./data/sales_curva_a.csv", delimiter=',', encoding='UTF-8', low_memory=False)
atacarejo

,date,item_nbr,unit_sales,price,day_of_year,month,day_of_week,day_of_month
0,2017-01-02,550,19.026,17.49,2,1,0,2
1,2017-01-03,550,8.822,17.49,3,1,1,3
2,2017-01-04,550,11.320,17.49,4,1,2,4
3,2017-01-05,550,6.828,17.49,5,1,3,5
4,2017-01-06,550,6.956,17.49,6,1,4,6
...,...,...,...,...,...,...,...,...
16975,2019-04-26,1001587,38.000,2.29,116,4,4,26
16976,2019-04-27,1001587,45.000,2.29,117,4,5,27
16977,2019-04-28,1001587,24.000,2.29,118,4,6,28
16978,2019-04-29,1001587,54.000,2.29,119,4,0,29


- Dataset contendo os feriados nacionais

In [3]:
feriados = pd.read_excel("./data/feriados_nacionais.xls")
feriados = feriados[:1263]
feriados

,Data,Dia da Semana,Feriado
0,2001-01-01 00:00:00,segunda-feira,Confraternização Universal
1,2001-02-26 00:00:00,segunda-feira,Carnaval
2,2001-02-27 00:00:00,terça-feira,Carnaval
3,2001-04-13 00:00:00,sexta-feira,Paixão de Cristo
4,2001-04-21 00:00:00,sábado,Tiradentes
...,...,...,...
1258,2099-09-07 00:00:00,segunda-feira,Independência do Brasil
1259,2099-10-12 00:00:00,segunda-feira,Nossa Sr.a Aparecida - Padroeira do Brasil
1260,2099-11-02 00:00:00,segunda-feira,Finados
1261,2099-11-15 00:00:00,domingo,Proclamação da República


- Filtrando apenas o intervalo de datas que contém no dataset de supermercado, no dataset de feriados

In [4]:
feriados['Data'] = pd.to_datetime(feriados['Data'], format='%d/%m/%Y')
feriados = feriados[(feriados['Data'] > '2017-01-01') & (feriados['Data'] < '2019-04-30')]
feriados = feriados['Data'].tolist()

In [21]:
import pickle
import json
import numpy as np

# Função para converter tipos que não são serializáveis em JSON
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int32, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

# Carregar os dados do arquivo pickle
with open("data/map_LSTM_varejo.pkl", "rb") as arquivo:
    dados = pickle.load(arquivo)

# Converter os dados para o formato JSON
dados_json = json.dumps(dados, cls=NumpyEncoder)

# Exibir os dados JSON
dados_json


'{"550": {"90": {"rodadas": {"rodada_0": [8.776993751525879, 8.798379898071289, 8.820717811584473, 8.844283103942871, 8.868340492248535, 8.892362594604492, 8.916036605834961, 8.939132690429688, 8.96150016784668, 8.983043670654297, 9.003707885742188, 9.02347183227539, 9.042352676391602, 9.060378074645996, 9.077489852905273, 9.09376335144043, 9.109285354614258, 9.124032020568848, 9.138036727905273, 9.151368141174316, 9.164021492004395, 9.176027297973633, 9.187430381774902, 9.198285102844238, 9.208585739135742, 9.218367576599121, 9.227656364440918, 9.23648452758789, 9.244797706604004, 9.252676010131836, 9.260234832763672, 9.267423629760742, 9.274231910705566, 9.280683517456055, 9.286805152893066, 9.29254150390625, 9.298027038574219, 9.303279876708984, 9.308247566223145, 9.31296157836914, 9.317437171936035, 9.321688652038574, 9.325671195983887, 9.329458236694336, 9.333089828491211, 9.33653736114502, 9.339807510375977, 9.342891693115234, 9.34583568572998, 9.348588943481445, 9.35122203826904

In [26]:
data = json.loads(dados_json)

# Extraindo as rodadas
rodadas = data["550"]["90"]["rodadas"]

# Extraindo valores de todas as rodadas em uma matriz
matriz = []
for rodada, valores_rodada in rodadas.items():
    matriz.append(valores_rodada)

# Imprimindo a matriz, com cada linha representando os valores de uma rodada
for linha in matriz:
    print(linha)

[8.776993751525879, 8.798379898071289, 8.820717811584473, 8.844283103942871, 8.868340492248535, 8.892362594604492, 8.916036605834961, 8.939132690429688, 8.96150016784668, 8.983043670654297, 9.003707885742188, 9.02347183227539, 9.042352676391602, 9.060378074645996, 9.077489852905273, 9.09376335144043, 9.109285354614258, 9.124032020568848, 9.138036727905273, 9.151368141174316, 9.164021492004395, 9.176027297973633, 9.187430381774902, 9.198285102844238, 9.208585739135742, 9.218367576599121, 9.227656364440918, 9.23648452758789, 9.244797706604004, 9.252676010131836, 9.260234832763672, 9.267423629760742, 9.274231910705566, 9.280683517456055, 9.286805152893066, 9.29254150390625, 9.298027038574219, 9.303279876708984, 9.308247566223145, 9.31296157836914, 9.317437171936035, 9.321688652038574, 9.325671195983887, 9.329458236694336, 9.333089828491211, 9.33653736114502, 9.339807510375977, 9.342891693115234, 9.34583568572998, 9.348588943481445, 9.351222038269043, 9.353724479675293, 9.356115341186523, 

3. Selecionando apenas as colunas necessárias no dataset de supermercado

In [5]:
atacarejo = atacarejo[['date','item_nbr','unit_sales','price','day_of_week']].copy()
atacarejo['date'] = pd.to_datetime(atacarejo['date'])
display(atacarejo.sample(5))
print(atacarejo.shape)

,date,item_nbr,unit_sales,price,day_of_week
10807,2018-09-13,47319,25.707,12.98,3
2779,2017-08-22,10146,180.170,1.79,1
5709,2018-09-09,20213,18.489,11.78,6
2937,2018-01-27,10146,44.835,2.77,5
3343,2019-03-09,10146,37.415,3.89,5


(16980, 5)


4. Adicionando a coluna feriado no dataset do supermercado

In [6]:
atacarejo['feriado'] = 0
atacarejo.loc[atacarejo['date'].isin(feriados), 'feriado'] = 1
display(atacarejo.sample(5))

,date,item_nbr,unit_sales,price,day_of_week,feriado
6349,2018-02-12,21869,89.815,3.65,0,1
3750,2017-12-22,11686,21.066,17.99,4,0
5955,2017-01-14,21869,91.935,3.78,5,0
435,2018-03-13,550,7.688,15.99,1,0
7014,2017-08-12,37082,11.600,11.98,5,0


5. Selecionando os id de todos os produtos e armazenando em id_produtos

In [7]:
id_produtos = atacarejo['item_nbr'].unique()
id_produtos = id_produtos.tolist()
print(id_produtos)

[550, 3124, 4022, 10146, 11686, 16766, 20213, 21869, 37082, 44516, 46756, 47289, 47319, 52740, 52849, 53419, 96560, 103893, 410861, 1001587]


In [13]:
mapeamento_dias = {
    0: 'Segunda-feira',
    1: 'Terça-feira',
    2: 'Quarta-feira',
    3: 'Quinta-feira',
    4: 'Sexta-feira',
    5: 'Sábado',
    6: 'Domingo'
}

#configuração da API
variables = load_dotenv()
variables = os.getenv('API_KEY')

genai.configure(api_key=variables)

#configuração do modelo
generation_config ={
    "candidate_count": 1,
    "temperature": 1.0,
}

#Datas do teste
data_inicio = pd.to_datetime('2017-01-02') # ano-mes-dia minimo 2017-01-02
data_fim = pd.to_datetime('2019-01-02') 

for produto in id_produtos[18:]:
    # Filtrando o produto e selecionando a data
    atacarejo2 = atacarejo[atacarejo['item_nbr'] == produto]
    atacarejo3 = atacarejo2[
                (atacarejo2['date'] >= data_inicio) & 
                (atacarejo2['date'] <= data_fim)
            ]

    df_exatos = atacarejo2[atacarejo2['date'] > data_fim]

    #listas de dados com os dados da serie temporal a ser analisada e com os dados futuros
    serie_temporal = atacarejo3['unit_sales'].to_list()
    serie_temporal = [round(x,3) for x in serie_temporal]

    serie_exata = df_exatos['unit_sales'].to_list()
    serie_exata = [round(x,3) for x in serie_exata]

    #Pegando os primeiros 7 dias da serie temporal
    dias_da_semana = atacarejo3.head(7)['day_of_week'].to_list()
    dias_encontrados = ""

    for i,dia in enumerate(dias_da_semana):
        dias_encontrados += f"  - Dia {i+1}: posiçao {i} ({mapeamento_dias[dia]});\n"

    #texto para os dias a serem previstos
    dias_da_semana_exato = df_exatos.head(7)['day_of_week'].tolist()

    dias_encontrados_exato = ""
    for i, dia in enumerate(dias_da_semana):
        dia_util = "Dia útil" if dia < 5 else "Fim de semana"
        dias_encontrados_exato += f"  - Dia {i+1} - ({mapeamento_dias[dia]}): {dia_util};\n"

    #feriados
    df_feriados = atacarejo3.reset_index(drop=True)
    df_feriados = df_feriados[df_feriados['feriado'] == 1]

    dias_feriados = ""
    for index, data in df_feriados.iterrows():
        dias_feriados += f"- Dia {index}: {mapeamento_dias[data['day_of_week']]}\n"
    

    #quantidade de dias da serie temporal
    quantidade_dias = atacarejo3.shape[0]

    #texto do prompt
    dias = 60

    prompt = f"""Você é um assistente de previsão de séries temporais encarregado de analisar dados de uma série temporal específica.
        
A série temporal tem dados de {quantidade_dias} periodo(s) consecutivos. Cada anotação da série temporal representa a incidência de um evento que ocorre a cada dia.

Por exemplo, uma semana nesta série temporal pode ser representado assim:
{atacarejo3['unit_sales'][:7].to_list()}

Seu objetivo é prever a incidência de um evento para os próximos N dias, levando em consideração não apenas os períodos anteriores, mas também o contexto geral.

Para fazer isso com precisão, leve em consideração os padrões sazonais, como picos em determinados períodos. Além disso, os padrões podem variar de acordo com o dia da semana ou eventos especiais (feriados).

Regras da Saída:
Após analisar os dados fornecidos e compreender os padrões de tráfego, gere uma previsão para os próximos N dias, com as seguintes regras:
- A saída deve ser uma lista contendo apenas os valores previstos, sem explicação adicional ou texto introdutório;
- Em hipótese alguma gere um código;
- Em hipótese alguma gere uma explicação do que você fez;
- Forneça apenas e exclusivamente um array contendo a quantidade de números solicitados.
- A previsão deve começar imediatamente após o último valor fornecido na série temporal.

Exemplo de Saída para N=7:
{atacarejo3['unit_sales'][:7].to_list()}

Instruções Adicionais:
- Padrões Semanais: Utilize os dados fornecidos para entender padrões sazonais, como picos de incidência em determinados períodos.
- Dia da Semana: O dia da semana também influencia a ocorrência de eventos. 
- Duração de um evento: A série temporal fornecida representa a ocorrência de um evento por dia.

Organização dos Dados:
- Cada dia corresponde a um valor na série temporal. Por exemplo:\n{dias_encontrados}
- E assim por diante.
- A cada valor, ocorre a transição para o dia seguinte.
- Períodos em que temos eventos especiais na série temporal:
    {dias_feriados}
Serie temporal a ser analisada:
{serie_temporal}

Contexto dos dias a serem previstos:\n{dias_encontrados_exato}- E assim por diante

Gere um array contendo os próximos {dias} (N={dias}) números da sequência:
"""
    print(prompt)
    #gerando a previsão
    model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")
    response = model.generate_content(prompt, generation_config=generation_config)

    resposta = response.text
    qtd_tokens = model.count_tokens(prompt)

    #smape
    resposta_float = [float(x.strip("] ")) for x in resposta.strip("[]\n").split(",")]

    tam = len(resposta_float)
    if tam > 60:
        tam = 60

    real = serie_exata[:tam]
    previsto = resposta_float[:tam]

    real, previsto = np.array(real), np.array(previsto)
    numerador = np.abs(real - previsto)
    denominador = np.abs(real) + np.abs(previsto)

    denominador = [denominador[i] if denominador[i]!=0 else 1 for i in range(len(denominador))]
    denominador = np.array(denominador)

    smape = round(np.mean(numerador / (denominador/2))*100, 2)

    #plotando o grafico
    os.makedirs(f"./img3", exist_ok=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(len(real))), y=real, mode='lines', name='Valor Real'))
    fig.add_trace(go.Scatter(x=list(range(len(previsto))), y=previsto, mode='lines', name='Valor Previsto'))

    fig.update_layout(
        title=f'Vendas por Dia - smape: {smape}%',
        xaxis_title='Dias',
        yaxis_title='Vendas',
        showlegend=True,
        colorway=['#1f77b4', '#ff7f0e'],
        height=600,
    )
    fig.write_image(f"./img3/{produto}.png", width=1400, height=600)


Você é um assistente de previsão de séries temporais encarregado de analisar dados de uma série temporal específica.
        
A série temporal tem dados de 731 periodo(s) consecutivos. Cada anotação da série temporal representa a incidência de um evento que ocorre a cada dia.

Por exemplo, uma semana nesta série temporal pode ser representado assim:
[67.0, 32.0, 44.0, 21.0, 34.0, 40.0, 27.0]

Seu objetivo é prever a incidência de um evento para os próximos N dias, levando em consideração não apenas os períodos anteriores, mas também o contexto geral.

Para fazer isso com precisão, leve em consideração os padrões sazonais, como picos em determinados períodos. Além disso, os padrões podem variar de acordo com o dia da semana ou eventos especiais (feriados).

Regras da Saída:
Após analisar os dados fornecidos e compreender os padrões de tráfego, gere uma previsão para os próximos N dias, com as seguintes regras:
- A saída deve ser uma lista contendo apenas os valores previstos, sem explic

ValueError: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.